In [2]:
# Import necessary libraries

import seaborn as sns
import matplotlib.pyplot as plt
import optuna
import yfinance as yf
import numpy as np
from vectorbtpro import *
import pandas as pd
vbt.settings.set_theme('dark')
import warnings
warnings.filterwarnings("ignore")
vbt.settings.plotting.use_resampler=(True)
vbt.settings['plotting']['layout']['width']=600
vbt.settings['plotting']['layout']['height']=200
from itertools import combinations, product


In [3]:
m1_data_path = '/Users/ericervin/Documents/Coding/data-repository/data/m1_data.h5'
m1_data = vbt.BinanceData.from_hdf(m1_data_path)
m1_data.data['BTCUSDT'].tail()
ema1 = 50
ema2 = 200
ema3 = 1000
resample_period = '1h'

# Resample data
m60_data = m1_data.resample(resample_period)
# display(m60_data.data['BTCUSDT'].tail())

# Simply grab the BTC data
btc = m1_data.select('BTCUSDT')
# display(btc.get().tail())

h4_data = m60_data.resample('4h')
data = h4_data.data['ETHUSDT']

helper funcs

In [7]:


def plot_charts(pf):
    fig = pf.plot(
        subplots=[
            "orders",
            "trade_pnl",
            "cum_returns",
            "drawdowns",
            "underwater",
            "gross_exposure",
        ],
        make_subplots_kwargs=dict(
            rows=3,
            cols=2,
            shared_xaxes=True,
        ),
        subplot_settings=dict(
            cum_returns=dict(pct_scale=True),
            underwater=dict(pct_scale=True),
        ),
    )
    fig["layout"].update(height=800, width=1200)
    fig.show()


def plot_charts_with_heatmap(pf, data):
    # Create the subplots layout
    fig = pf.plot(
        subplots=[
            "orders",
            "trade_pnl",
            "cum_returns",
            "drawdowns",
            "underwater",
            "gross_exposure",
        ],
        make_subplots_kwargs=dict(
            rows=4,  # Adding an extra row for the heatmap
            cols=2,
            shared_xaxes=True,
            specs=[
                [{}, {}],
                [{}, {}],
                [{}, {}],
                [{"secondary_y": True}, None],
            ],  # Secondary y-axis enabled in the last row, first column
        ),
        subplot_settings=dict(
            cum_returns=dict(pct_scale=True),
            underwater=dict(pct_scale=True),
        ),
    )

    fig["layout"].update(height=800, width=1200)

    # Create the heatmap overlay
    pf.value.vbt.overlay_with_heatmap(data["Labels"], title="Regime Overlay")

    fig.add_trace(
        pf.value.vbt.overlay_with_heatmap(data["Labels"], title="Regime Overlay").data[0],
        row=4,
        col=1,
    )
    fig.add_trace(
        pf.value.vbt.overlay_with_heatmap(data["Labels"], title="Regime Overlay").data[1],
        row=4,
        col=1,
        secondary_y=True,
    )
    fig.show()
    


In [8]:
def backtest_ma(data, short_window, long_window):
    # Apply the strategy
    short_ma = data['Close'].rolling(window=short_window).mean()
    long_ma = data['Close'].rolling(window=long_window).mean() # may want to use this 
    
    # Conditions 

    up_trend = short_ma > long_ma
    down_trend = ~up_trend
    
    entries = up_trend
    short_entries = down_trend
    
    pf = vbt.Portfolio.from_signals(
        close=data['Close'],
        entries=entries,
        short_entries=short_entries,
        fees = 0.00075,
    )
    
    return pf

def backtest_ma_twitchy(data, short_window, long_window):
    # Apply the strategy
    short_ma = data['Close'].rolling(window=short_window).mean()
    long_ma = data['Close'].rolling(window=long_window).mean() # may want to use this 
    
    # Conditions 

    up_trend = short_ma > long_ma
    down_trend = short_ma < long_ma
    
    entries = up_trend
    exits = down_trend | data['Close'] < short_ma # may want to change to long_ma
    short_entries = down_trend
    short_exits = up_trend | data['Close'] > short_ma # may want to change to long_ma
    
    pf = vbt.Portfolio.from_signals(
        close=data['Close'],
        entries=entries,
        exits=exits,
        short_entries=short_entries,
        short_exits=short_exits,
        fees = 0.00075,
    )
    
    return pf

def objective_ma(trial, data):
    short_window = trial.suggest_int('short_window', 1, 50)
    long_window = trial.suggest_int('long_window', 20, 200)
    
    if short_window >= long_window:
        return float('inf')
    
    pf = backtest_ma(data, short_window, long_window)
    
    return -pf.total_return



data['Return'] = data['Close'].pct_change()

# Optimize the strategy
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective_ma(trial, data), n_trials=1000)

# Print the best parameters
best_params = study.best_params
print('Best parameters: ', best_params)
print('Best value: ', study.best_value)

# Backtest with the best parameters
best_pf = backtest_ma(
    data,
    best_params['short_window'],
    best_params['long_window'],
)

# Compare it to a buy and hold strategy
buy_hold_pf = vbt.Portfolio.from_holding(data['Close'])

comparison = pd.concat({
    'Optimized': best_pf.stats(),
    'Buy and Hold': buy_hold_pf.stats()
}, axis=1)

print(comparison)
plot_charts(best_pf)


[I 2024-08-23 15:58:14,733] A new study created in memory with name: no-name-aad01cfc-e7af-4583-851c-9401f608ccc0
[I 2024-08-23 15:58:14,761] Trial 0 finished with value: -0.3925862716039154 and parameters: {'short_window': 31, 'long_window': 105}. Best is trial 0 with value: -0.3925862716039154.
[I 2024-08-23 15:58:14,777] Trial 1 finished with value: -3.6930799909633585 and parameters: {'short_window': 31, 'long_window': 79}. Best is trial 1 with value: -3.6930799909633585.
[I 2024-08-23 15:58:14,791] Trial 2 finished with value: 0.7881392027841 and parameters: {'short_window': 9, 'long_window': 30}. Best is trial 1 with value: -3.6930799909633585.
[I 2024-08-23 15:58:14,802] Trial 3 finished with value: -10.194620674216731 and parameters: {'short_window': 17, 'long_window': 151}. Best is trial 3 with value: -10.194620674216731.
[I 2024-08-23 15:58:14,812] Trial 4 finished with value: -6.253204793944238 and parameters: {'short_window': 42, 'long_window': 188}. Best is trial 3 with va

Best parameters:  {'short_window': 8, 'long_window': 196}
Best value:  -55.1484177088113
                                             Optimized  \
Start Index                  2019-09-08 16:00:00+00:00   
End Index                    2024-07-12 20:00:00+00:00   
Total Duration                      1769 days 08:00:00   
Start Value                                      100.0   
Min Value                                    92.636002   
Max Value                                  6585.779634   
End Value                                  5614.841771   
Total Return [%]                           5514.841771   
Benchmark Return [%]                       2026.363014   
Position Coverage [%]                        95.506782   
Max Gross Exposure [%]                      136.248066   
Max Drawdown [%]                             54.673635   
Max Drawdown Duration                698 days 20:00:00   
Total Orders                                        89   
Total Fees Paid                          

In [6]:
# The `best_pf.value` attribute contains equity curve of the portfolio
best_pf.value.vbt.plot().show()

# Let's evaluate periods of weaker performance
best_pf.plot_underwater(pct_scale=True).show()
best_pf.plot_drawdowns().show()